In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

# Constants
hbar = 0.19732  # GeV·fm
sigma = 0.184   # GeV^2
cf = 4 / 3
alpha_fixed = 0.3
gs = np.sqrt(4 * np.pi * alpha_fixed)
xmin, xmax = 0.01, 30.0
interval = 2001
mc = 1.35
mcr = mc / 2.0

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Debye mass
def mD_fixed(T):
    return np.sqrt(4 * np.pi * alpha_fixed) * T

# Potential function
def screened_cornell_potential_fixed(T):
    mD = mD_fixed(T)
    def V(r):
        r = torch.where(r < 0.001, torch.tensor(0.001, device=device), r)
        V_coul = -cf * alpha_fixed * torch.exp(-mD * r) / r
        V_conf = sigma * (1 - torch.exp(-mD * r)) / mD
        return V_coul + V_conf
    return V

# Laplacian operator
def laplacian(N, dr):
    main_diag = -2 * torch.ones(N, device=device)
    off_diag = torch.ones(N - 1, device=device)
    L = torch.diag(main_diag) + torch.diag(off_diag, diagonal=1) + torch.diag(off_diag, diagonal=-1)
    return L / dr**2

# Solve Schrödinger equation using torch
def solve_schrodinger(V_func, mass, hbar, xmin, xmax, interval, n=0, l=0):
    r = torch.linspace(xmin, xmax, interval, device=device)
    dr = r[1] - r[0]
    centrifugal = l * (l + 1) * hbar**2 / (2 * mass * r**2)
    V_eff = V_func(r) + centrifugal
    U = torch.diag(V_eff)
    L = laplacian(interval, dr)
    H = -hbar**2 / (2 * mass) * L + U
    eigvals, eigvecs = torch.linalg.eigh(H)
    return eigvals[n], eigvecs[:, n], r

# Corrected BE function
def corrected_binding_energy(eigen_energy, mD, gs, sigma=0.184):
    shift = eigen_energy - 2.0 * sigma / mD + mD * gs**2 / (4.0 * np.pi)
    return 2.0 * torch.abs(shift)

# Temperature list
T_list = torch.linspace(0.155, 0.5, 20, device=device)

# State groups
groups = {
    "1s": [(0, 0, '1s')],
    "2s_2p": [(1, 0, '2s'), (1, 1, '2p')],
    "3s_3p_3d": [(2, 0, '3s'), (2, 1, '3p'), (2, 2, '3d')]
}

# Output directory
output_dir = "binding_vs_temperature_torch"
os.makedirs(output_dir, exist_ok=True)

# Main loop
for group_name, state_list in groups.items():
    plt.figure(figsize=(10, 6))
    for n, l, label in state_list:
        binding_vs_T = []
        for T in T_list:
            T_val = T.item()
            mD = mD_fixed(T_val)
            V_T = screened_cornell_potential_fixed(T_val)
            try:
                E, _, _ = solve_schrodinger(V_T, mcr, hbar, xmin, xmax, interval, n=n, l=l)
                eigen_energy = E - 2.0 * mc
                BE = corrected_binding_energy(eigen_energy, mD, gs, sigma)
                BE_val = BE.item()
            except:
                BE_val = 0.0
            binding_vs_T.append((T_val, BE_val))
            print(f"T = {T_val:.3f} GeV, state {label}: BE = {BE_val:.4f} GeV")

        # Save data
        file_path = os.path.join(output_dir, f"{label}_binding_vs_T.dat")
        with open(file_path, 'w') as f:
            f.write("# T [GeV]   Binding Energy [GeV]\n")
            for T_val, BE_val in binding_vs_T:
                f.write(f"{T_val:.5f}   {BE_val:.6f}\n")

        # Plotting
        T_vals, BE_vals = zip(*binding_vs_T)
        plt.plot(np.array(T_vals)*1000, np.array(BE_vals), label=label, linewidth=2)

    plt.xlabel("Temperature T [MeV]")
    plt.ylabel("Binding Energy [GeV]")
    plt.title(f"Binding Energy vs Temperature: {group_name.replace('_', ', ')}")
    plt.grid(True, alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plot_path = os.path.join(output_dir, f"binding_vs_T_{group_name}.png")
    plt.savefig(plot_path, dpi=300)
    plt.close()

print(f"\nAll results saved in directory: {os.path.abspath(output_dir)}")


T = 0.155 GeV, state 1s: BE = 10.6144 GeV
T = 0.173 GeV, state 1s: BE = 10.3105 GeV
T = 0.191 GeV, state 1s: BE = 10.0554 GeV
T = 0.209 GeV, state 1s: BE = 9.8364 GeV
T = 0.228 GeV, state 1s: BE = 9.6451 GeV
T = 0.246 GeV, state 1s: BE = 9.4751 GeV
T = 0.264 GeV, state 1s: BE = 9.3224 GeV
T = 0.282 GeV, state 1s: BE = 9.1835 GeV
T = 0.300 GeV, state 1s: BE = 9.0559 GeV
T = 0.318 GeV, state 1s: BE = 8.9376 GeV
T = 0.337 GeV, state 1s: BE = 8.8274 GeV
T = 0.355 GeV, state 1s: BE = 8.7238 GeV
T = 0.373 GeV, state 1s: BE = 8.6260 GeV
T = 0.391 GeV, state 1s: BE = 8.5332 GeV
T = 0.409 GeV, state 1s: BE = 8.4448 GeV
T = 0.427 GeV, state 1s: BE = 8.3601 GeV
T = 0.446 GeV, state 1s: BE = 8.2789 GeV
T = 0.464 GeV, state 1s: BE = 8.2006 GeV
T = 0.482 GeV, state 1s: BE = 8.1249 GeV
T = 0.500 GeV, state 1s: BE = 8.0516 GeV
T = 0.155 GeV, state 2s: BE = 7.9534 GeV
T = 0.173 GeV, state 2s: BE = 7.6567 GeV
T = 0.191 GeV, state 2s: BE = 7.4090 GeV
T = 0.209 GeV, state 2s: BE = 7.1980 GeV
T = 0.228 GeV